# Smol model recognize lift

## Load data


In [ ]:
import os
import random
from shutil import copy2
from sklearn.model_selection import train_test_split

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Change directory to your project folder
os.chdir('/content/drive/My Drive/LYFTCAR/')

Mounted at /content/drive


In [ ]:


# Define paths and parameters
input_folder = 'lift_images'  # Folder containing subfolders "0" and "1"
output_train_folder = 'lift_images/processed_images/train'
output_test_folder = 'lift_images/processed_images/test'
test_size = 0.2  # Percentage of data to use for testing
random_seed = 42  # For reproducibility

# Create all output directories for train and test sets
for label in ['0', '1']:
    os.makedirs(os.path.join(output_train_folder, label), exist_ok=True)
    os.makedirs(os.path.join(output_test_folder, label), exist_ok=True)

# Gather all image file paths from both classes
image_paths = []
labels = []

for label in ['0', '1']:
    folder_path = os.path.join(input_folder, label)
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.jpg', '.png', '.jpeg')):  # Check for valid image files
            image_paths.append(os.path.join(folder_path, filename))
            labels.append(label)

# Split the data into train and test sets
train_paths, test_paths, train_labels, test_labels = train_test_split(
    image_paths, labels, test_size=test_size, random_state=random_seed
)

# Function to copy images to the corresponding folder
def copy_images(image_paths, labels, output_folder):
    for img_path, label in zip(image_paths, labels):
        # Ensure the output directory for the label exists
        label_folder = os.path.join(output_folder, label)
        os.makedirs(label_folder, exist_ok=True)

        # Copy the image to the corresponding output folder
        output_path = os.path.join(label_folder, os.path.basename(img_path))
        copy2(img_path, output_path)

# Copy training images
print("Copying training images...")
copy_images(train_paths, train_labels, output_train_folder)

# Copy test images
print("Copying test images...")
copy_images(test_paths, test_labels, output_test_folder)

print("Image splitting into train and test sets completed!")


Mounted at /content/drive
Copying training images...
Copying test images...
Image splitting into train and test sets completed!


In [ ]:
!pip install tensorflow

## Model

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Step 1: Create the lightweight CNN model
def create_lightweight_cnn(input_shape=(120, 160, 3), num_classes=2):  # Update num_classes to 2 for binary classification
    model = models.Sequential()

    # First convolutional layer (16 filters, kernel size 3x3)
    model.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))  # Max pooling layer

    # Second convolutional layer (32 filters, kernel size 3x3)
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))  # Max pooling layer

    # Third convolutional layer (32 filters, kernel size 3x3)
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))  # Max pooling layer

    # Flatten the output to feed into dense layers
    model.add(layers.Flatten())

    # Reduce dense layer size to 32 units
    model.add(layers.Dense(32, activation='relu'))

    # Output layer with softmax for classification (num_classes)
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model


## Training

In [ ]:
# Step 2: Set up directories for training and testing images
train_dir = 'lift_images/processed_images/test'
test_dir = 'lift_images/processed_images/train'

# Step 3: Create ImageDataGenerator for loading images
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,      # Normalize pixel values to 0-1
    rotation_range=20,      # Optional: rotate images for augmentation
    width_shift_range=0.2,  # Optional: shift width for augmentation
    height_shift_range=0.2, # Optional: shift height for augmentation
    shear_range=0.2,        # Optional: shear augmentation
    zoom_range=0.2,         # Optional: zoom augmentation
    horizontal_flip=True,   # Optional: flip images horizontally
    fill_mode='nearest'     # Filling strategy for new pixels after augmentation
)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)  # Only normalization for test data

# Step 4: Load images from directories and apply the transformations
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(120, 160),  # Resize images to the target size
    batch_size=32,           # Number of images per batch
    class_mode='categorical' # Assuming binary classification; change to 'binary' if required
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(120, 160),  # Resize test images
    batch_size=32,           # Batch size for testing
    class_mode='categorical' # Same as for training
)

# Step 5: Compile the model
input_shape = (120, 160, 3)  # Adjust based on image size
num_classes = 2  # Update to 2 for binary classification

model = create_lightweight_cnn(input_shape, num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Step 6: Summarize the model
model.summary()

# Step 7: Train the model
history = model.fit(
    train_generator,
    epochs=100,                    # Adjust number of epochs as needed
    validation_data=test_generator # Validate with the test data
)

# Step 8: Evaluate the model on the test set
loss, accuracy = model.evaluate(test_generator)
print(f"Test accuracy: {accuracy:.2f}")

model_save_path = 'lift_images/models/lightweight_cnn_model.h5'  # Define the path to save the model
model.save(model_save_path)  # Save the model
print(f"Model saved to: {model_save_path}")


Found 438 images belonging to 2 classes.
Found 1748 images belonging to 2 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 118, 158, 16)        │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 59, 79, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 57, 77, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 28, 38, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 26, 36, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 13, 18, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 7488)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 32)                  │         239,648 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │              66 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 254,050 (992.38 KB)

...

 Non-trainable params: 0 (0.00 B)

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


14/14 ━━━━━━━━━━━━━━━━━━━━ 778s 55s/step - accuracy: 0.5101 - loss: 0.6806 - val_accuracy: 0.8816 - val_loss: 0.4697
Epoch 2/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 36s 695ms/step - accuracy: 0.7263 - loss: 0.5403 - val_accuracy: 0.7283 - val_loss: 0.4764
Epoch 3/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 11s 674ms/step - accuracy: 0.7430 - loss: 0.5045 - val_accuracy: 0.9354 - val_loss: 0.2621
Epoch 4/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 21s 657ms/step - accuracy: 0.7987 - loss: 0.4387 - val_accuracy: 0.9348 - val_loss: 0.1919
Epoch 5/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 23s 917ms/step - accuracy: 0.8416 - loss: 0.3415 - val_accuracy: 0.9634 - val_loss: 0.1173
Epoch 6/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 22s 944ms/step - accuracy: 0.8495 - loss: 0.3590 - val_accuracy: 0.8656 - val_loss: 0.3505
Epoch 7/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 11s 582ms/step - accuracy: 0.9243 - loss: 0.2348 - val_accuracy: 0.8375 - val_loss: 0.4670
Epoch 8/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 11s 667ms/step - accuracy: 0.9122 - loss: 0.2464 - val_accuracy:

Test accuracy: 0.99
Model saved to: lift_images/models/lightweight_cnn_model.h5


In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf

def load_and_preprocess_image(image_path, target_size=(120, 160)):
    """Load an image, resize it, and normalize pixel values."""
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Image not found at path: {image_path}")
    img = cv2.resize(img, target_size)  # Resize to the target size
    img = img / 255.0  # Normalize pixel values to [0, 1]
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

def inference_with_accuracy(model, test_images_dir):
    """Perform inference on images in the specified directory and calculate accuracy."""

    # Walk through the directory tree and collect image paths and labels
    test_image_paths = []
    ground_truth_labels = []

    for root, _, files in os.walk(test_images_dir):
        for file in files:
            if file.endswith(('.jpg', '.png', '.jpeg')):
                image_path = os.path.join(root, file)
                label = int(os.path.basename(root))  # Extract label from folder name (0 or 1)
                test_image_paths.append(image_path)
                ground_truth_labels.append(label)

    predictions = []
    correct_predictions = 0

    # Make predictions for each image
    for image_path, true_label in zip(test_image_paths, ground_truth_labels):
        processed_image = load_and_preprocess_image(image_path)
        prediction = model.predict(processed_image)  # Get predictions
        predicted_class = np.argmax(prediction, axis=1)[0]  # Convert to class label
        predictions.append((image_path, predicted_class, true_label))  # Store image path, predicted class, and true label

        # Check if the prediction is correct
        if predicted_class == true_label:
            correct_predictions += 1

    # Calculate accuracy
    accuracy = correct_predictions / len(test_image_paths) if test_image_paths else 0.0
    return predictions, accuracy

def print_predictions_with_accuracy(predictions, accuracy):
    """Print the predictions and accuracy in a readable format."""
    for img_path, pred_class, true_label in predictions:
        print(f"Image: {img_path} -> Predicted Class: {pred_class} (True Class: {true_label})")
    print(f"\nAccuracy: {accuracy * 100:.2f}%")

# Example usage
if __name__ == "__main__":
    # Load the trained model
    model = tf.keras.models.load_model('lift_images/models/lightweight_cnn_model.h5')  # Replace with your model path

    # Specify the directory containing test images
    test_images_dir = 'lift_images/processed_images/test/'  # Replace with the actual directory path

    # Perform inference and calculate accuracy
    predictions, accuracy = inference_with_accuracy(model, test_images_dir)

    # Print the results
    print_predictions_with_accuracy(predictions, accuracy)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 270ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━

## Convert to tflite


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model
with open("model.tflite", "wb") as f:
    f.write(tflite_model)